In [713]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_squared_error
import datetime
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics

In [714]:
gameTable = pd.read_csv('C://Users//Michael//Desktop//Springboard//Capstone1//dataCleaning//cleanedData//cleanedGameData.csv')
dvoaTable = pd.read_csv('C://Users//Michael//Desktop//Springboard//Capstone1//dataCleaning//cleanedData//cleanedDvoaData.csv', encoding='windows-1252')
pffTable = pd.read_csv('C://Users//Michael//Desktop//Springboard//Capstone1//dataCleaning//cleanedData//cleanedPffData.csv')

In [715]:
# First, turn date cols into dt objects
gameTable['date'] = pd.to_datetime(gameTable['date'])
pffTable['date'] = pd.to_datetime(pffTable['date'])

In [716]:
# Create a table of running averages for pff grades and game scores. Use previous 5-game averages for a team's variables
rollAvgPff = pffTable.set_index('date')

rollAvgPff['avgPf'] = rollAvgPff.groupby('team')['pf'].apply(lambda x: x.shift(1).rolling(window=5).mean())
rollAvgPff['avgPa'] = rollAvgPff.groupby('team')['pa'].apply(lambda x: x.shift(1).rolling(window=5).mean())
rollAvgPff['avgOvr'] = rollAvgPff.groupby('team')['overall'].apply(lambda x: x.shift(1).rolling(window=5).mean())
rollAvgPff['avgOff'] = rollAvgPff.groupby('team')['off'].apply(lambda x: x.shift(1).rolling(window=5).mean())
rollAvgPff['avgPass'] = rollAvgPff.groupby('team')['pass'].apply(lambda x: x.shift(1).rolling(window=5).mean())
rollAvgPff['avgPblk'] = rollAvgPff.groupby('team')['pblk'].apply(lambda x: x.shift(1).rolling(window=5).mean())
rollAvgPff['avgRecv'] = rollAvgPff.groupby('team')['recv'].apply(lambda x: x.shift(1).rolling(window=5).mean())
rollAvgPff['avgRun'] = rollAvgPff.groupby('team')['run'].apply(lambda x: x.shift(1).rolling(window=5).mean())
rollAvgPff['avgRblk'] = rollAvgPff.groupby('team')['rblk'].apply(lambda x: x.shift(1).rolling(window=5).mean())
rollAvgPff['avgDef'] = rollAvgPff.groupby('team')['def'].apply(lambda x: x.shift(1).rolling(window=5).mean())
rollAvgPff['avgRdef'] = rollAvgPff.groupby('team')['rdef'].apply(lambda x: x.shift(1).rolling(window=5).mean())
rollAvgPff['avgTack'] = rollAvgPff.groupby('team')['tack'].apply(lambda x: x.shift(1).rolling(window=5).mean())
rollAvgPff['avgPrsh'] = rollAvgPff.groupby('team')['prsh'].apply(lambda x: x.shift(1).rolling(window=5).mean())
rollAvgPff['avgCov'] = rollAvgPff.groupby('team')['cov'].apply(lambda x: x.shift(1).rolling(window=5).mean())

rollAvgPff = rollAvgPff.fillna(method='bfill')
rollAvgPff = rollAvgPff.reset_index()

In [717]:
# Need to make new rollingAvgPff tables that have all dates covered... Dates in pff table are off by one or two days
# Strategy is to create new df's that have all dates, then join the normal pff table to each of them by date,
# then merge all the tables together. All dates will be in the final table
pffB1 = rollAvgPff.copy()
pffB1['date'] = pffB1['date'] - datetime.timedelta(days=1)
pffB2 = rollAvgPff.copy()
pffB2['date'] = pffB2['date'] - datetime.timedelta(days=2)
pffF1 = rollAvgPff.copy()
pffF1['date'] = pffF1['date'] + datetime.timedelta(days=1)
pffF2 = rollAvgPff.copy()
pffF2['date'] = pffF2['date'] + datetime.timedelta(days=2)

In [718]:
# Now join pffTable to each new df
t0 = pd.merge(gameTable,rollAvgPff,how='inner',left_on=['date','awayTeam'], right_on=['date','team'])
t1 = pd.merge(gameTable,pffB1,how='inner',left_on=['date','awayTeam'], right_on=['date','team'])
t2 = pd.merge(gameTable,pffB2,how='inner',left_on=['date','awayTeam'], right_on=['date','team'])
t3 = pd.merge(gameTable,pffF1,how='inner',left_on=['date','awayTeam'], right_on=['date','team'])
t4 = pd.merge(gameTable,pffF2,how='inner',left_on=['date','awayTeam'], right_on=['date','team'])
t5 = pd.merge(gameTable,rollAvgPff,how='inner',left_on=['date','homeTeam'], right_on=['date','team'])
t6 = pd.merge(gameTable,pffB1,how='inner',left_on=['date','homeTeam'], right_on=['date','team'])
t7 = pd.merge(gameTable,pffB2,how='inner',left_on=['date','homeTeam'], right_on=['date','team'])
t8 = pd.merge(gameTable,pffF1,how='inner',left_on=['date','homeTeam'], right_on=['date','team'])
t9 = pd.merge(gameTable,pffF2,how='inner',left_on=['date','homeTeam'], right_on=['date','team'])

In [719]:
awayTable = t0.append([t1,t2,t3,t4],ignore_index=True)
homeTable = t5.append([t6,t7,t8,t9],ignore_index=True)

In [720]:
awayTable['homeAway'] = 'away'
homeTable['homeAway'] = 'home'

In [721]:
awayTable.columns

Index(['gameId', 'date', 'awayTeam', 'awayScore', 'homeTeam', 'homeScore',
       'totalScore', 'overUnder', 'result', 'stadium', 'roof', 'surface',
       'vegasLine', 'referee', 'umpire', 'headLinesman', 'lineJudge',
       'backJudge', 'sideJudge', 'fieldJudge', 'season', 'week', 'team', 'opp',
       'time', 'w', 'pf', 'pa', 'overall', 'off', 'pass', 'pblk', 'recv',
       'run', 'rblk', 'def', 'rdef', 'tack', 'prsh', 'cov', 'spec', 'lateGame',
       'year', 'avgPf', 'avgPa', 'avgOvr', 'avgOff', 'avgPass', 'avgPblk',
       'avgRecv', 'avgRun', 'avgRblk', 'avgDef', 'avgRdef', 'avgTack',
       'avgPrsh', 'avgCov', 'homeAway'],
      dtype='object')

In [722]:
awayTable = awayTable[['gameId', 'date','season', 'week','team','homeAway',
       'stadium', 'roof', 'surface', 'avgPf', 'avgPa', 'avgOvr', 'avgOff',
       'avgPass', 'avgPblk', 'avgRecv', 'avgRun', 'avgRblk', 'avgDef', 'avgRdef',
       'avgPrsh', 'avgCov','pf','pa','totalScore', 'overUnder', 'result']]
homeTable = homeTable[['gameId', 'date','season', 'week','team','homeAway',
       'stadium', 'roof', 'surface', 'avgPf', 'avgPa', 'avgOvr', 'avgOff',
       'avgPass', 'avgPblk', 'avgRecv', 'avgRun', 'avgRblk', 'avgDef', 'avgRdef',
       'avgPrsh', 'avgCov','pf','pa','totalScore', 'overUnder', 'result']]


In [723]:
# Transform week from dvoa table into correct format for analysis (add one)
dvoaTable['week'] = dvoaTable['week'] + 1

In [724]:
# Next, join these tables with DVOA table
awayTable = awayTable.merge(dvoaTable, on=['season','week','team'])
homeTable = homeTable.merge(dvoaTable, on=['season','week','team'])

In [725]:
# We only want the team's offensive and total grade columns and the team's opponent's
# defensive/total grade cols
# We can make two copies of both tables: awayOff, awayDef, homeOff, homeDef. Then join
# awayOff with homeDef, and join homeOff with awayDef and predict team totals for each game
# after transforming variables by combining columns.
awayOff = awayTable[['gameId', 'date', 'season', 'week', 'team', 'homeAway', 'stadium',
       'roof', 'surface', 'avgPf', 'avgOvr', 'avgOff', 'avgPass',
       'avgPblk', 'avgRecv', 'avgRun', 'avgRblk', 'totalDvoa', 'offDvoa', 'stDvoa',
       'overUnder', 'pf', 'pa', 'totalScore']]
awayDef = awayTable[['gameId', 'date', 'season', 'week', 'team', 'stadium',
       'roof', 'surface', 'avgPa', 'avgOvr', 'avgDef', 'avgRdef',
       'avgPrsh', 'avgCov', 'totalDvoa', 'defDvoa', 'totalScore']]
homeOff = homeTable[['gameId', 'date', 'season', 'week', 'team', 'homeAway', 'stadium',
       'roof', 'surface', 'avgPf', 'avgOvr', 'avgOff', 'avgPass',
       'avgPblk', 'avgRecv', 'avgRun', 'avgRblk', 'totalDvoa', 'offDvoa', 'stDvoa',
       'overUnder', 'pf', 'pa', 'totalScore']]
homeDef = homeTable[['gameId', 'date', 'season', 'week', 'team', 'stadium',
       'roof', 'surface', 'avgPa', 'avgOvr', 'avgDef', 'avgRdef',
       'avgPrsh', 'avgCov', 'totalDvoa', 'defDvoa', 'totalScore']]

In [726]:
awayMatchup = awayOff.merge(homeDef, on=['gameId', 'date', 'season', 'week', 'stadium', 'roof', 'surface', 'totalScore'])
homeMatchup = homeOff.merge(awayDef, on=['gameId', 'date', 'season', 'week', 'stadium', 'roof', 'surface', 'totalScore'])

In [727]:
# Now we will create new columns for these two tables by combining matchups
# Ex) add awayMatchup['avgOff'] + awayMatchup['avgDef'] ('avgDef' is the 
# away team's opponent's average defensive grade)
# to turn these two columns into one

awayMatchup['ptsMatchup'] = awayMatchup['avgPf'] + awayMatchup['avgPa']
awayMatchup['offMatchup'] = awayMatchup['avgOff'] - awayMatchup['avgDef']
awayMatchup['ovrMatchup'] = awayMatchup['avgOvr_x'] - awayMatchup['avgOvr_y']
awayMatchup['passMatchup'] = awayMatchup['avgPass'] - awayMatchup['avgCov']
awayMatchup['pblkMatchup'] = awayMatchup['avgPblk'] - awayMatchup['avgPrsh']
awayMatchup['runMatchup'] = awayMatchup['avgRun'] - awayMatchup['avgRdef']
awayMatchup['offDvoaMatchup'] = awayMatchup['offDvoa'] - awayMatchup['defDvoa']
awayMatchup['totalDvoaMatchup'] = awayMatchup['totalDvoa_x'] - awayMatchup['totalDvoa_y']

homeMatchup['ptsMatchup'] = homeMatchup['avgPf'] + homeMatchup['avgPa']
homeMatchup['offMatchup'] = homeMatchup['avgOff'] - homeMatchup['avgDef']
homeMatchup['ovrMatchup'] = homeMatchup['avgOvr_x'] - homeMatchup['avgOvr_y']
homeMatchup['passMatchup'] = homeMatchup['avgPass'] - homeMatchup['avgCov']
homeMatchup['pblkMatchup'] = homeMatchup['avgPblk'] - homeMatchup['avgPrsh']
homeMatchup['runMatchup'] = homeMatchup['avgRun'] - homeMatchup['avgRdef']
homeMatchup['offDvoaMatchup'] = homeMatchup['offDvoa'] - homeMatchup['defDvoa']
homeMatchup['totalDvoaMatchup'] = homeMatchup['totalDvoa_x'] - homeMatchup['totalDvoa_y']

In [728]:
# Put 'points for' as the last column

cols = list(awayMatchup.columns)
cols
cols.pop(21)
cols.append('pf')

In [729]:
awayMatchup = awayMatchup[cols]
homeMatchup = homeMatchup[cols]

In [730]:
X_train_away, X_test_away, y_train_away, y_test_away = train_test_split(awayMatchup.iloc[:,:-1],awayMatchup.iloc[:,-1:], test_size=0.2,random_state=981)

In [731]:
X_train_away.shape

(2494, 40)

In [732]:
# Now we want to predict points for (pf) for the homeMatchup/awayMatchup tables
# Setup table in way to do so:
ID_col = ['gameId']
target_col = ['pf']
cat_cols = ['roof','surface']
num_cols= ['ptsMatchup', 'offMatchup', 'ovrMatchup', 'passMatchup', 'pblkMatchup',
           'runMatchup', 'totalDvoaMatchup', 'offDvoaMatchup', 'avgRecv','overUnder']
other_col= ['date','season','week','stadium','team_x','team_y','homeAway','avgPf',
            'avgOvr_x', 'avgOff', 'avgPass', 'avgPblk','avgRun', 'avgRblk',
            'totalDvoa_x', 'offDvoa','pa', 'totalScore','avgPa', 'avgOvr_y',
            'avgDef', 'avgRdef', 'avgPrsh', 'avgCov', 'totalDvoa_y', 'defDvoa']

In [733]:
# Create label encoders for categorical variables and split the data set
# to train & test, further split the train data set to Train and Validate

#create label encoders for categorical features
for var in cat_cols:
    number = LabelEncoder()
    X_train_away[var] = number.fit_transform(X_train_away[var].astype('str'))

for var in cat_cols:
    number = LabelEncoder()
    X_test_away[var] = number.fit_transform(X_test_away[var].astype('str'))

In [734]:
# Pass the imputed and dummy (missing values flags) variables into the
# modeling process. Use random forest to predict the class
features=list(set(list(X_train_away.columns))-set(ID_col)-set(target_col)-set(other_col))

In [735]:
X_train_away = X_train_away[list(features)]
X_test_away= X_test_away[list(features)]

In [736]:
X_train_away

,ovrMatchup,passMatchup,runMatchup,stDvoa,overUnder,ptsMatchup,offMatchup,avgRecv,surface,roof,offDvoaMatchup,pblkMatchup,totalDvoaMatchup
847,-1.50,3.02,-20.90,0.002,46.5,50.8,-1.66,72.84,0,1,-0.126,9.96,-0.171
2191,10.82,19.96,-1.14,-0.013,47.5,47.4,16.68,73.70,1,0,0.188,11.36,0.245
1424,0.90,19.00,2.86,-0.015,47.5,52.6,15.62,71.58,1,1,0.032,2.60,-0.002
274,0.76,11.70,2.36,0.019,44.5,47.2,7.02,60.32,1,0,-0.026,14.28,0.002
2980,4.16,11.00,-11.64,-0.049,37.5,42.8,4.06,71.46,0,1,0.196,6.80,0.423
1480,13.22,9.32,-5.00,0.023,45.5,48.2,13.82,72.70,1,0,-0.200,11.78,0.319
880,1.18,-0.78,10.58,0.004,47.0,43.4,4.68,70.48,1,0,0.196,-12.16,0.025
3041,1.70,-2.68,2.52,-0.014,37.0,48.0,9.98,61.60,0,1,0.134,17.72,0.149
365,-4.60,9.80,3.94,0.072,53.0,51.8,5.08,71.78,1,0,0.153,-5.26,0.073
1781,-4.42,-7.08,2.86,-0.069,43.5,40.8,3.80,64.06,0,1,-0.142,12.54,-0.373


In [754]:
rf = RandomForestRegressor(n_estimators=600, random_state=0, max_depth=6, min_impurity_decrease=0.6)
rf.fit(X_train_away, y_train_away)
y_pred_away = rf.predict(X_test_away)
print(mean_squared_error(y_test_away, y_pred_away))

C:\Python34\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


83.5191688538


In [749]:
print(rf.get_params)

<bound method RandomForestRegressor.get_params of RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=7,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=600, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)>


In [738]:
# Now repeat the process for homeMatchup table:
X_train_home, X_test_home, y_train_home, y_test_home = train_test_split(homeMatchup.iloc[:,:-1],homeMatchup.iloc[:,-1:], test_size=0.2,random_state=981)

for var in cat_cols:
    number = LabelEncoder()
    X_train_home[var] = number.fit_transform(X_train_home[var].astype('str'))

for var in cat_cols:
    number = LabelEncoder()
    X_test_home[var] = number.fit_transform(X_test_home[var].astype('str'))
    
features=list(set(list(X_train_home.columns))-set(ID_col)-set(target_col)-set(other_col))

X_train_home = X_train_home[list(features)]
X_test_home= X_test_home[list(features)]

In [742]:
X_train_home

,ovrMatchup,passMatchup,runMatchup,stDvoa,overUnder,ptsMatchup,offMatchup,avgRecv,surface,roof,offDvoaMatchup,pblkMatchup,totalDvoaMatchup
847,1.50,-6.640000e+00,-2.92,0.002,46.5,44.8,1.14,66.76,0,1,-0.080,9.62,0.171
2191,-10.82,-5.460000e+00,0.74,0.061,47.5,44.2,0.94,65.74,1,0,-0.016,12.20,-0.245
1424,-0.90,2.238000e+01,-4.24,0.008,47.5,49.0,12.58,71.24,1,1,0.101,-12.38,0.002
274,-0.76,9.540000e+00,-14.36,0.051,44.5,44.6,3.42,71.72,1,0,0.001,5.64,-0.002
2980,-4.16,1.860000e+00,-11.02,0.051,37.5,36.0,-2.08,61.38,0,1,0.147,0.84,-0.423
1480,-13.22,-1.428000e+01,-16.22,0.003,45.5,38.4,-8.08,65.52,1,0,0.274,9.84,-0.319
880,-1.18,8.200000e+00,-3.36,-0.029,47.0,53.6,3.52,68.52,1,0,0.043,-0.36,-0.025
3041,-1.70,-3.500000e+00,0.10,0.006,37.0,45.4,6.74,69.14,0,1,0.265,5.42,-0.149
365,4.60,1.086000e+01,6.06,0.052,53.0,57.0,15.50,70.34,1,0,0.082,12.96,-0.073
1781,4.42,2.054000e+01,-11.88,-0.026,43.5,52.4,10.18,63.48,0,1,-0.137,23.04,0.373


In [743]:
rf = RandomForestRegressor(n_estimators=100, random_state=0, max_depth=6)
rf.fit(X_train_home, y_train_home)
y_pred_home = rf.predict(X_test_home)
print(mean_squared_error(y_test_home, y_pred_home))

C:\Python34\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


99.2682391676


In [700]:
testDf = pd.DataFrame(X_test_away)
testDf['awayScore'] = y_test_away
testDf['awayPrediction'] = y_pred_away
testDf2 = pd.DataFrame(X_test_home)
testDf2['homeScore'] = y_test_home
testDf2['homePrediction'] = y_pred_home

In [701]:
colsAway = list(testDf.columns)
colsAway.pop(-1)
colsAway.pop(-1)
colsAway.pop(8)
colsAway.pop(8)
#colsAway
colsHome = list(testDf2.columns)
colsHome.pop(-1)
colsHome.pop(-1)
colsHome.pop(8)
colsHome.pop(8)
#colsHome

'roof'

In [702]:
awayPredDf = awayMatchup.merge(testDf, on=colsAway)
homePredDf = homeMatchup.merge(testDf2, on=colsHome)

In [703]:
awayHomeMerge = awayPredDf.merge(homePredDf, on=['gameId','date','season','week'])
#awayHomeMerge.columns

In [704]:
awayHomeMerge = awayHomeMerge[['gameId', 'date', 'season', 'week','awayPrediction', 'homePrediction']]
awayHomeMerge.head()

,gameId,date,season,week,awayPrediction,homePrediction
0,201612240sea,2016-12-24,2016,16,18.718423,24.312932
1,201612110mia,2016-12-11,2016,14,20.120170,25.089417
2,201711190htx,2017-11-19,2017,11,18.925606,23.726720
3,201611200min,2016-11-20,2016,11,19.491166,23.738973
4,201711050car,2017-11-05,2017,9,20.071268,22.031652


In [705]:
finalDf = gameTable.merge(awayHomeMerge, on=['gameId', 'date'])
finalDf['totalPrediction'] = finalDf['awayPrediction'] + finalDf['homePrediction']
#finalDf.columns

In [706]:
finalDf = finalDf[['gameId', 'date', 'season', 'week', 'awayTeam', 'awayPrediction', 'awayScore',
                   'homeTeam','homePrediction', 'homeScore', 'overUnder', 'result', 'totalPrediction', 'totalScore']]

In [707]:
undersDf = finalDf[finalDf['totalPrediction'] <= finalDf['overUnder'] - 5]
undersDf['result'].value_counts()

over     6
under    2
push     1
Name: result, dtype: int64

In [708]:
undersDf = finalDf[finalDf['totalPrediction'] <= finalDf['overUnder'] - 3]
undersDf['result'].value_counts()

under    21
over     20
push      2
Name: result, dtype: int64

In [709]:
undersDf = finalDf[finalDf['totalPrediction'] <= finalDf['overUnder']]
undersDf['result'].value_counts()

over     135
under    132
push       5
Name: result, dtype: int64

In [710]:
oversDf = finalDf[finalDf['totalPrediction'] >= finalDf['overUnder'] + 5]
oversDf['result'].value_counts()

under    12
over      9
push      1
Name: result, dtype: int64

In [711]:
oversDf = finalDf[finalDf['totalPrediction'] >= finalDf['overUnder'] + 3]
oversDf['result'].value_counts()

under    44
over     42
push      2
Name: result, dtype: int64

In [712]:
oversDf = finalDf[finalDf['totalPrediction'] >= finalDf['overUnder']]
oversDf['result'].value_counts()

over     178
under    169
push       5
Name: result, dtype: int64

In [460]:
oversDf[oversDf['season'] == 2018]

,gameId,date,season,week,awayTeam,awayPrediction,awayScore,homeTeam,homePrediction,homeScore,overUnder,result,totalPrediction,totalScore
0,201811110kan,2018-11-11,2018,10,Arizona Cardinals,18.754386,14,Kansas City Chiefs,35.175439,26,49.5,under,53.929825,40
106,201810140mia,2018-10-14,2018,6,Chicago Bears,24.631579,28,Miami Dolphins,27.000000,31,40.5,over,51.631579,59
476,201812160ram,2018-12-16,2018,15,Philadelphia Eagles,22.578947,30,Los Angeles Rams,34.000000,23,52.5,over,56.578947,53
539,201812160sfo,2018-12-16,2018,15,Seattle Seahawks,28.114035,23,San Francisco 49ers,22.017544,26,44.0,over,50.131579,49
605,201809230jax,2018-09-23,2018,3,Tennessee Titans,16.157895,9,Jacksonville Jaguars,27.008772,6,39.0,under,43.166667,15


In [999]:
# Now join awayTable and homeTable to gameTable
mainDf = gameTable.merge(awayTable, on=['gameId','date','awayTeam']).merge(homeTable, on=['gameId','date','season','week','homeTeam'])

In [1000]:
mainDf.columns

Index(['gameId', 'date', 'awayTeam', 'awayScore', 'homeTeam', 'homeScore',
       'totalScore', 'overUnder', 'result', 'stadium', 'roof', 'surface',
       'vegasLine', 'referee', 'umpire', 'headLinesman', 'lineJudge',
       'backJudge', 'sideJudge', 'fieldJudge', 'season', 'week', 'awayAvgPf',
       'awayAvgPa', 'awayAvgOvr', 'awayAvgOff', 'awayAvgPass', 'awayAvgPblk',
       'awayAvgRecv', 'awayAvgRun', 'awayAvgRblk', 'awayAvgDef', 'awayAvgRdef',
       'awayAvgTack', 'awayAvgPrsh', 'awayAvgCov', 'awayTotalDvoa',
       'awayOffDvoa', 'awayOffRank', 'awayDefDvoa', 'awayDefRank',
       'awayStDvoa', 'awayStRank', 'homeAvgPf', 'homeAvgPa', 'homeAvgOvr',
       'homeAvgOff', 'homeAvgPass', 'homeAvgPblk', 'homeAvgRecv', 'homeAvgRun',
       'homeAvgRblk', 'homeAvgDef', 'homeAvgRdef', 'homeAvgTack',
       'homeAvgPrsh', 'homeAvgCov', 'homeTotalDvoa', 'homeOffDvoa',
       'homeOffRank', 'homeDefDvoa', 'homeDefRank', 'homeStDvoa',
       'homeStRank'],
      dtype='object')

In [1001]:
# Take out cols we don't want
mainDf = mainDf.loc[:,['gameId', 'date','season', 'week', 'stadium', 'roof', 'surface',
       'awayTeam','awayAvgPf','awayAvgPa', 'awayAvgOvr', 'awayAvgOff', 'awayAvgPass', 'awayAvgPblk',
       'awayAvgRecv', 'awayAvgRun', 'awayAvgRblk', 'awayAvgDef', 'awayAvgRdef','awayAvgTack', 'awayAvgPrsh',
       'awayAvgCov','awayTotalDvoa', 'awayOffDvoa', 'awayDefDvoa', 'awayStDvoa',
       'homeTeam','homeAvgPf', 'homeAvgPa', 'homeAvgOvr', 'homeAvgOff', 'homeAvgPass', 'homeAvgPblk',
       'homeAvgRecv', 'homeAvgRun', 'homeAvgRblk', 'homeAvgDef', 'homeAvgRdef', 'homeAvgTack',
       'homeAvgPrsh', 'homeAvgCov', 'homeTotalDvoa', 'homeOffDvoa', 'homeDefDvoa', 'homeStDvoa',
       'overUnder', 'result', 'totalScore']]

In [1002]:
mainDf = mainDf[mainDf['result'] != 'push']

In [1006]:
mainDf = mainDf[mainDf['week'] >= 3]

In [1013]:
mainDf

,gameId,date,season,week,stadium,roof,surface,awayTeam,awayAvgPf,awayAvgPa,...,homeAvgTack,homeAvgPrsh,homeAvgCov,homeTotalDvoa,homeOffDvoa,homeDefDvoa,homeStDvoa,overUnder,result,totalScore
0,201811110kan,2018-11-11,2018,10,Arrowhead Stadium,outdoors,grass,Arizona Cardinals,18.0,25.0,...,67.56,72.30,73.30,0.372,0.333,0.065,0.105,49.5,under,40
1,201011210kan,2010-11-21,2010,11,Arrowhead Stadium,outdoors,grass,Arizona Cardinals,23.4,28.6,...,73.40,68.10,61.98,0.212,0.121,-0.096,-0.004,45.0,under,44
3,200901100car,2009-01-10,2008,19,Bank of America Stadium,outdoors,grass,Arizona Cardinals,23.8,27.4,...,73.18,68.84,60.68,0.198,0.186,0.013,0.025,49.5,under,46
4,201501030car,2015-01-03,2014,18,Bank of America Stadium,outdoors,grass,Arizona Cardinals,14.0,20.8,...,75.96,69.66,74.46,-0.124,-0.059,0.008,-0.058,37.5,over,43
5,201012190car,2010-12-19,2010,15,Bank of America Stadium,outdoors,grass,Arizona Cardinals,17.2,25.2,...,63.46,64.16,59.02,-0.383,-0.358,0.026,0.002,37.5,under,31
6,200810260car,2008-10-26,2008,8,Bank of America Stadium,outdoors,grass,Arizona Cardinals,30.8,26.2,...,71.54,65.90,71.06,0.058,0.054,-0.024,-0.020,43.0,over,50
7,201610300car,2016-10-30,2016,8,Bank of America Stadium,outdoors,grass,Arizona Cardinals,19.6,16.0,...,65.78,62.62,60.00,-0.106,0.000,0.036,-0.070,45.5,over,50
8,200912140sfo,2009-12-14,2009,14,Candlestick Park,outdoors,grass,Arizona Cardinals,28.0,18.2,...,66.24,68.72,60.78,-0.016,-0.100,-0.076,0.008,45.0,under,33
9,201212300sfo,2012-12-30,2012,17,Candlestick Park,outdoors,grass,Arizona Cardinals,14.8,26.8,...,78.56,59.94,66.76,0.348,0.185,-0.171,-0.008,39.5,over,40
10,201101020sfo,2011-01-02,2010,17,Candlestick Park,outdoors,grass,Arizona Cardinals,18.8,20.8,...,70.54,65.04,57.46,-0.111,-0.064,0.028,-0.019,39.0,over,45


In [1459]:
X_train, X_test, y_train, y_test = train_test_split(mainDf.iloc[:,:-1],mainDf.iloc[:,-1:], test_size=0.2,randomrrr_state=555)

In [1460]:
ID_col = ['gameId']
target_col = ['result','totalScore']
cat_cols = ['roof','surface']
num_cols= ['awayAvgPf','awayAvgPa', 'awayAvgOvr', 'awayAvgOff', 'awayAvgPass', 'awayAvgPblk',
           'awayAvgRecv', 'awayAvgRun', 'awayAvgRblk', 'awayAvgDef', 'awayAvgRdef','awayAvgTack', 'awayAvgPrsh',
           'awayAvgCov','awayTotalDvoa','awayOffDvoa','awayDefDvoa','awayStDvoa','homeAvgPf', 'homeAvgPa', 'homeAvgOvr', 'homeAvgOff', 'homeAvgPass', 'homeAvgPblk',
           'homeAvgRecv', 'homeAvgRun', 'homeAvgRblk', 'homeAvgDef', 'homeAvgRdef', 'homeAvgTack',
           'homeAvgPrsh', 'homeAvgCov','homeTotalDvoa','homeOffDvoa','homeDefDvoa','homeStDvoa','overUnder']
other_col= ['date','season','week','stadium','awayTeam','homeTeam'] #Test and Train Data set identifier

In [1461]:
# Create label encoders for categorical variables and split the data set
# to train & test, further split the train data set to Train and Validate

#create label encoders for categorical features
for var in cat_cols:
    number = LabelEncoder()
    X_train[var] = number.fit_transform(X_train[var].astype('str'))

for var in cat_cols:
    number = LabelEncoder()
    X_test[var] = number.fit_transform(X_test[var].astype('str'))

In [1462]:
# Pass the imputed and dummy (missing values flags) variables into the
# modeling process. Use random forest to predict the class
features=list(set(list(X_train.columns))-set(ID_col)-set(target_col)-set(other_col))

In [1463]:
X_train = X_train[list(features)]
X_test= X_test[list(features)]

In [1464]:
#random.seed(100)
rf = RandomForestRegressor(n_estimators=200, randomrrr_state=0)
rf.fit(X_train, y_train)
#x_test = x_test.reshape(-1,1)
y_pred = rf.predict(X_test)

C:\Python34\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [1465]:
print(mean_squared_error(y_test, y_pred))

192.217477542


In [1466]:
# 80 - 203.645811688

In [1467]:
testDf = pd.DataFrame(X_test)
testDf['totalScore'] = y_test
testDf['predictedScore'] = y_pred
#testDf.columns

In [1468]:
testDf = testDf[['surface', 'roof', 'homeAvgRblk', 'homeAvgDef', 'homeAvgRdef', 'awayAvgRblk', 'homeAvgOff',
       'awayAvgTack', 'homeAvgCov', 'homeAvgTack','awayAvgCov',
       'awayTotalDvoa', 'homeAvgRecv', 'awayOffDvoa','homeAvgOvr',
       'awayAvgPblk', 'awayAvgPrsh', 'homeOffDvoa', 'awayAvgPa', 'awayAvgOff',
       'awayAvgDef', 'homeAvgPass', 'homeAvgPa', 'homeStDvoa', 'awayAvgOvr',
       'awayAvgPass', 'homeAvgPf', 'homeAvgPrsh', 'homeDefDvoa', 'homeAvgRun',
       'awayAvgPf', 'awayAvgRecv', 'homeTotalDvoa', 'awayDefDvoa',
       'awayStDvoa', 'awayAvgRdef', 'homeAvgPblk', 'awayAvgRun','overUnder',
       'predictedScore','totalScore']]
#testDf

In [1469]:
oversDf = testDf[testDf['predictedScore'] >= testDf['overUnder'] + 2.5]

In [1470]:
oversDf.loc[oversDf.totalScore > oversDf.overUnder, 'result'] = 'over'
oversDf.loc[oversDf.totalScore == oversDf.overUnder, 'result'] = 'push'
oversDf.loc[oversDf.totalScore < oversDf.overUnder, 'result'] = 'under'
#oversDf

In [1471]:
oversDf['result'].value_counts()

# rs = 0 - 73/60 - 54.8%
# rs = 1 - 70/61 - 53.4%
# rs = 2 - 64/62 - 50.8%
# rs = 23 - 59/56 - 51.3%
# rs = 28 - 76/68 - 52.77%
# rs = 119 - 76/70 - 52.05%
# rs = 251 - 67/62 - 51.9%
# rs = 1190 - 67/64 - 51.14%
# rs = 1232 - 77/54 - 58.8%
# rs = 875 - 57/63 - 47.5%
# rs = 877 - 59/65 - 47.6%
# rs = 337 - 57/61 - 48.3%
# rs = 420 - 53/55 - 49.07%
# rs = 555 - 60/61 - 49.6%
#TOTAL - 51.5% 

under    61
over     60
Name: result, dtype: int64

In [1452]:
# Now do the same for unders
undersDf = testDf[testDf['predictedScore'] <= testDf['overUnder'] - 3]
undersDf.loc[undersDf.totalScore > undersDf.overUnder, 'result'] = 'over'
undersDf.loc[undersDf.totalScore == undersDf.overUnder, 'result'] = 'push'
undersDf.loc[undersDf.totalScore < undersDf.overUnder, 'result'] = 'under'
#undersDf

In [1453]:
undersDf['result'].value_counts()
# rs = 1 - 24/21 - 53.33%
# rs = 2 - 33/33 - 50.0%
# rs = 23 - 20/29 - 40.8%
# rs = 28 - 18/17 - 51.4%
# rs = 119 - 29/20 - 59.18%
# rs = 251 - 26/32 - 44.8%
# rs = 50%

over     27
under    25
Name: result, dtype: int64

In [1454]:
cols = list(oversDf.columns)
cols.pop(0)
cols.pop(0)
cols.pop(-3)
#cols

'predictedScore'

In [1455]:
oDf = mainDf.merge(oversDf, on=cols)

In [1456]:
#oDf.columns

In [1457]:
oDf = oDf[['gameId', 'date', 'season', 'week', 'stadium', 'roof_x', 'surface_x',
       'awayTeam','homeTeam', 'overUnder','predictedScore', 'totalScore','result']]

In [1458]:
oDf[(oDf['season'] >= 2018)]

,gameId,date,season,week,stadium,roof_x,surface_x,awayTeam,homeTeam,overUnder,predictedScore,totalScore,result
0,201812300sea,2018-12-30,2018,17,CenturyLink Field,outdoors,turf,Arizona Cardinals,Seattle Seahawks,39.5,44.935,51,over
7,201810140min,2018-10-14,2018,6,U.S. Bank Stadium,dome,turf,Arizona Cardinals,Minnesota Vikings,43.5,46.610,44,over
18,201811110nyj,2018-11-11,2018,10,MetLife Stadium,outdoors,turf,Buffalo Bills,New York Jets,38.0,40.930,51,over
24,201812020nyg,2018-12-02,2018,13,MetLife Stadium,outdoors,turf,Chicago Bears,New York Giants,44.0,51.915,57,over
30,201812090sdg,2018-12-09,2018,14,StubHub Center,outdoors,grass,Cincinnati Bengals,Los Angeles Chargers,48.5,51.355,47,under
33,201901120ram,2019-01-12,2018,19,Los Angeles Memorial Coliseum,outdoors,grass,Dallas Cowboys,Los Angeles Rams,48.0,52.145,52,over
55,201812300htx,2018-12-30,2018,17,NRG Stadium,dome,turf,Jacksonville Jaguars,Houston Texans,39.5,43.780,23,under
63,201809230ram,2018-09-23,2018,3,Los Angeles Memorial Coliseum,outdoors,grass,Los Angeles Chargers,Los Angeles Rams,49.0,53.085,58,over
64,201812090chi,2018-12-09,2018,14,Soldier Field,outdoors,grass,Los Angeles Rams,Chicago Bears,51.0,53.860,21,under
66,201812300buf,2018-12-30,2018,17,New Era Field,outdoors,turf,Miami Dolphins,Buffalo Bills,39.5,46.615,59,over


In [967]:
# Create a table of running averages for pff grades and game scores. Use previous 5-game averages for a team's variables
rollAvgPff = pffTable.set_index('date')

rollAvgPff['avgPf'] = rollAvgPff.groupby('team')['pf'].apply(lambda x: x.shift(1).rolling(window=5).mean())
rollAvgPff['avgPa'] = rollAvgPff.groupby('team')['pa'].apply(lambda x: x.shift(1).rolling(window=5).mean())
rollAvgPff['avgOvr'] = rollAvgPff.groupby('team')['overall'].apply(lambda x: x.shift(1).rolling(window=5).mean())
rollAvgPff['avgOff'] = rollAvgPff.groupby('team')['off'].apply(lambda x: x.shift(1).rolling(window=5).mean())
rollAvgPff['avgPass'] = rollAvgPff.groupby('team')['pass'].apply(lambda x: x.shift(1).rolling(window=5).mean())
rollAvgPff['avgPblk'] = rollAvgPff.groupby('team')['pblk'].apply(lambda x: x.shift(1).rolling(window=5).mean())
rollAvgPff['avgRecv'] = rollAvgPff.groupby('team')['recv'].apply(lambda x: x.shift(1).rolling(window=5).mean())
rollAvgPff['avgRun'] = rollAvgPff.groupby('team')['run'].apply(lambda x: x.shift(1).rolling(window=5).mean())
rollAvgPff['avgRblk'] = rollAvgPff.groupby('team')['rblk'].apply(lambda x: x.shift(1).rolling(window=5).mean())
rollAvgPff['avgDef'] = rollAvgPff.groupby('team')['def'].apply(lambda x: x.shift(1).rolling(window=5).mean())
rollAvgPff['avgRdef'] = rollAvgPff.groupby('team')['rdef'].apply(lambda x: x.shift(1).rolling(window=5).mean())
rollAvgPff['avgTack'] = rollAvgPff.groupby('team')['tack'].apply(lambda x: x.shift(1).rolling(window=5).mean())
rollAvgPff['avgPrsh'] = rollAvgPff.groupby('team')['prsh'].apply(lambda x: x.shift(1).rolling(window=5).mean())
rollAvgPff['avgCov'] = rollAvgPff.groupby('team')['cov'].apply(lambda x: x.shift(1).rolling(window=5).mean())

rollAvgPff = rollAvgPff.fillna(method='bfill')
rollAvgPff = rollAvgPff.reset_index()

In [968]:
rollAvgPff = rollAvgPff.fillna(method='bfill')

In [969]:
rollAvgPff = rollAvgPff.reset_index()

In [970]:
rollAvgPff

,date,season,week,team,opp,time,w,pf,pa,overall,...,avgPass,avgPblk,avgRecv,avgRun,avgRblk,avgDef,avgRdef,avgTack,avgPrsh,avgCov
0,2006-09-10,2006,1,Arizona Cardinals,San Francisco 49ers,11:00am,W,34,27,72.5,...,51.84,69.70,68.20,52.08,63.54,63.96,67.04,55.04,57.74,63.80
1,2006-09-17,2006,2,Arizona Cardinals,Seattle Seahawks,11:00am,L,10,21,48.0,...,51.84,69.70,68.20,52.08,63.54,63.96,67.04,55.04,57.74,63.80
2,2006-09-24,2006,3,Arizona Cardinals,Los Angeles Rams,11:00am,L,14,16,65.6,...,51.84,69.70,68.20,52.08,63.54,63.96,67.04,55.04,57.74,63.80
3,2006-10-01,2006,4,Arizona Cardinals,Atlanta Falcons,8:00am,L,10,32,64.1,...,51.84,69.70,68.20,52.08,63.54,63.96,67.04,55.04,57.74,63.80
4,2006-10-08,2006,5,Arizona Cardinals,Kansas City Chiefs,8:00am,L,20,23,65.6,...,51.84,69.70,68.20,52.08,63.54,63.96,67.04,55.04,57.74,63.80
5,2006-10-16,2006,6,Arizona Cardinals,Chicago Bears,12:00am,L,23,24,66.0,...,51.84,69.70,68.20,52.08,63.54,63.96,67.04,55.04,57.74,63.80
6,2006-10-22,2006,7,Arizona Cardinals,Oakland Raiders,9:00am,L,9,22,61.5,...,46.14,70.56,65.98,49.48,62.84,66.74,66.26,60.46,61.24,66.12
7,2006-10-29,2006,8,Arizona Cardinals,Green Bay Packers,7:00am,L,14,31,55.0,...,49.28,72.66,66.46,45.30,61.52,70.40,73.48,69.36,60.34,66.30
8,2006-11-12,2006,10,Arizona Cardinals,Dallas Cowboys,7:00am,L,10,27,51.0,...,50.14,71.50,63.66,44.00,63.08,67.72,68.66,72.56,60.50,64.62
9,2006-11-19,2006,11,Arizona Cardinals,Detroit Lions,7:00am,W,17,10,73.1,...,53.44,66.40,62.76,46.28,61.02,63.22,64.66,62.58,60.70,59.92
